# Further Time Series Analysis

In [ ]:
from datetime import datetime
import pandas as pd
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
%matplotlib inline 

## Analysing Existing Time Series Data

To begin this notebook, we will work with existing time series data from the US Energy Information Administration which lists Brent crude oil prices over several decades. Prices are given in dollars per barrel, and dates are in the format DD/MM/YYYY.

First, load the data from a remote CSV file into a Pandas DataFrame. Again, we will need a custom date parser to extract the dates. We will use the dates as the row index for the DataFrame. Note that we tell Pandas to parse the *Date* column and we indicate that the format has the "day first".

In [ ]:
df = pd.read_csv("http://mlg.ucd.ie/modules/COMP41680/oil.csv", 
    parse_dates=['Date'], index_col="Date", dayfirst=True)

First, sort the data by date (i.e. by the DataFrame's index):

In [ ]:
df = df.sort_index()
df.tail(5)

We can look at the date range for the series:

In [ ]:
dmin, dmax = df.index.min(), df.index.max()
dmin, dmax

First, we will plot the entire time series:

In [ ]:
ax = df["Price"].plot(figsize=(13,7), color="darkgreen", fontsize=14)
ax.set_xlabel("Date", fontsize=14)
ax.set_ylabel("Oil Price Per Barrel ($)", fontsize=14);
# ensure the y-axis starts at 0
ax.set_ylim(0)
# only include the exact time period of the data
ax.set_xlim(dmin, dmax);

We will focus on the data from 2000 onwards. We can use slicing to access that portion of the series.

In [ ]:
df2 = df.loc["2000-01-01":]
# check the range
dmin2, dmax2 = df2.index.min(), df2.index.max()
dmin2, dmax2

In [ ]:
ax = df2["Price"].plot(figsize=(13,7), color="darkgreen", fontsize=14)
ax.set_xlabel("Date", fontsize=14)
ax.set_ylabel("Oil Price Per Barrel ($)", fontsize=14);
# ensure the y-axis starts at 0
ax.set_ylim(0)
# only include the exact time period of the data
ax.set_xlim(dmin2, dmax2);

Currently the data is based on weekly oil prices. We can roll this up to a monthly level, by taking the mean of the values in each month using the Pandas *groupby()* function.

In [ ]:
groups = df2.groupby(pd.Grouper(freq='M'))
df3 = groups.agg({'Price':'mean'})
df3.head(10)

In [ ]:
ax = df3["Price"].plot(figsize=(13,7), color="darkgreen", fontsize=14)
ax.set_xlabel("Date", fontsize=14)
ax.set_ylabel("Oil Price Per Barrel ($)", fontsize=14);
# ensure the y-axis starts at 0
ax.set_ylim(0)
# only include the exact time period of the data
ax.set_xlim(dmin2, dmax2);

When exactly were the peaks in the time series?

In [ ]:
df3.sort_values(by="Price", ascending=False).head(10)

And where were the troughs in the time series?

In [ ]:
df3.sort_values(by="Price").head(10)

## Moving Averages

To get a better view of the trend in oil prices, we will use a 6-month (half yearly) rolling mean to smooth the data:

In [ ]:
rm6 = df3.rolling(6).mean()
rm6.head(10)

Note that we do not have values for the first 5 months, as the 6-month mean can only be calculated at the third month. We can remove these from the data.

In [ ]:
rm6 = rm6.loc["2000-06-30":]
rm6.head()

In [ ]:
ax = rm6["Price"].plot(figsize=(13,7), color="darkgreen", fontsize=14)
ax.set_xlabel("Date", fontsize=14)
ax.set_ylabel("Oil Price Per Barrel ($)", fontsize=14);
# ensure the y-axis starts at 0
ax.set_ylim(0)
# only include the exact time period of the rolling average data
ax.set_xlim("2000-06-30", dmax2);

We could also generate series of rolling averages, with different window sizes, and compare it to the raw monthly data.

In [ ]:
raw = df3["Price"]
raw.name = "Raw Data"

In [ ]:
# compute the 6 month rolling average
rm6 = raw.rolling(6).mean()
rm6.name = "6-month Average"

In [ ]:
# compute the 24 month rolling average
rm24 = raw.rolling(24).mean()
rm24.name = "24-month Average"

In [ ]:
# merge the three series to create a new DataFrame
dfb = pd.concat([raw, rm6, rm24], axis=1)
dfb.head(10)

In [ ]:
ax = dfb.plot(figsize=(14, 7), fontsize=14, color=["lightgray","orange", "blue"])
ax.set_xlabel("Date", fontsize=14)
ax.set_ylabel("Oil Price Per Barrel ($)", fontsize=14);
ax.set_ylim(0);

## Creating Time Series from Data

In some cases we will have data containing rows, where each row has a date or timestamp. In this case we might have multiple rows which have the same date (e.g. multiple visits to a website on the same day, multiple purchases made on the same day). It can be useful to create a time series from this data based on counts for the dates.

To demonstrate this, we will work with a dataset containing website log data. Here each row indicates a visit to a different website at a different time. We might be interested in the number of visits or "hits" for the website each day. We could create a time series showing this data.

First, load the data from a remote CSV file into a Pandas DataFrame. In this case we will parse the field *Timestamp*, and set it as the index of our DataFrame.

In [ ]:
df_web = pd.read_csv("http://mlg.ucd.ie/modules/COMP41680/website-log.csv", 
                 parse_dates=["Timestamp"], index_col="Timestamp")
df_web.head(10)

If we want to count the number of hits per day, we need to resample the time series to a daily frequency and then look at the frequency per day using the *size()* function:

In [ ]:
# resample by day (D) and use size to count the rows for each day
daily_visits = df_web.resample('D').size()
daily_visits

In [ ]:
# create the plot
p = daily_visits.plot(figsize=(9, 4.5), fontsize=13, 
    style="-o", color="darkblue", markerfacecolor="red", markersize=8)
# customise the axis labels
p.set_xlabel("Date", fontsize=13)
p.set_ylabel("Number of Visits", fontsize=13);